### Bronze Layer: Ingest pit_stops.json

Load raw JSON (multiline) from landing zone and save as parquet in bronze/raw layer.

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import current_timestamp, lit

In [ ]:
from formula1.formula1_constants import landing_folder_path, raw_folder_path

##### Step 1 - Define schema and read JSON from landing zone

In [ ]:
pit_stops_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("stop", StringType(), True),
    StructField("lap", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

In [ ]:
pit_stops_df = spark.read \
    .option("multiLine", True) \
    .schema(pit_stops_schema) \
    .json(f"{landing_folder_path}/pit_stops.json")

##### Step 2 - Add ingestion metadata

In [ ]:
pit_stops_df = pit_stops_df \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("data_source", lit(v_data_source))

##### Step 3 - Write to bronze/raw layer as parquet

In [ ]:
pit_stops_df.write.mode("overwrite").parquet(f"{raw_folder_path}/pit_stops")